# Example of semi-supervised gan for mnist

Based on https://machinelearningmastery.com/semi-supervised-generative-adversarial-network/

In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

%matplotlib notebook

Using TensorFlow backend.


In [2]:
# custom activation function
def custom_activation(output):
    logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result

In [3]:
# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # image input
    in_image = Input(shape=in_shape)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output layer nodes
    fe = Dense(n_classes)(fe)
    # supervised output
    c_out_layer = Activation('softmax')(fe)
    # define and compile supervised discriminator model
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    # unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
    # define and compile unsupervised discriminator model
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return d_model, c_model

In [4]:
# define the standalone generator model
def define_generator(latent_dim):
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # define model
    model = Model(in_lat, out_layer)
    return model

In [5]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect image output from generator as input to discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and outputting a classification
    model = Model(g_model.input, gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
# load the images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    print(X.shape, trainy.shape)
    return [X, trainy]

In [7]:
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), n_per_class)
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list)

In [8]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    z_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = z_input.reshape(n_samples, latent_dim)
    return z_input

In [10]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict(z_input)
    # create class labels
    y = zeros((n_samples, 1))
    return images, y

In [11]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(100):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # evaluate the classifier model
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'g_model_%04d.h5' % (step+1)
    g_model.save(filename2)
    # save the classifier model
    filename3 = 'c_model_%04d.h5' % (step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [12]:
# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100):
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset)

In [13]:
# size of the latent space
latent_dim = 100
# create the discriminator models
d_model, c_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(60000, 28, 28, 1) (60000,)
(100, 28, 28, 1) (100,)
n_epochs=20, n_batch=100, 1/2=50, b/e=600, steps=12000
Instructions for updating:
Use tf.cast instead.


C:\Users\Serge\Anaconda3\envs\nlp\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, c[2.310,14], d[0.098,2.399], g[0.095]
>2, c[2.301,14], d[0.093,2.398], g[0.095]
>3, c[2.282,20], d[0.089,2.399], g[0.095]
>4, c[2.307,14], d[0.086,2.399], g[0.095]
>5, c[2.295,4], d[0.083,2.399], g[0.096]
>6, c[2.276,20], d[0.081,2.397], g[0.096]
>7, c[2.223,28], d[0.079,2.391], g[0.097]
>8, c[2.230,30], d[0.080,2.383], g[0.098]
>9, c[2.247,16], d[0.079,2.377], g[0.098]
>10, c[2.223,22], d[0.075,2.377], g[0.098]
>11, c[2.216,22], d[0.066,2.379], g[0.098]
>12, c[2.171,28], d[0.058,2.376], g[0.099]
>13, c[2.131,36], d[0.047,2.363], g[0.101]
>14, c[2.082,40], d[0.036,2.344], g[0.103]
>15, c[2.157,22], d[0.028,2.331], g[0.105]
>16, c[2.097,18], d[0.020,2.326], g[0.106]
>17, c[1.940,48], d[0.017,2.335], g[0.109]
>18, c[1.825,70], d[0.021,2.307], g[0.122]
>19, c[1.858,56], d[0.036,2.157], g[0.150]
>20, c[2.096,20], d[0.081,2.037], g[0.157]
>21, c[1.833,44], d[0.082,2.021], g[0.155]
>22, c[1.703,42], d[0.036,1.960], g[0.170]
>23, c[1.832,34], d[0.024,1.817], g[0.206]
>24, c[1.721,46], d[0

>192, c[0.121,98], d[0.826,0.687], g[1.173]
>193, c[0.128,98], d[0.859,0.823], g[1.169]
>194, c[0.142,98], d[0.957,0.915], g[1.383]
>195, c[0.152,100], d[0.692,0.798], g[1.368]
>196, c[0.208,96], d[0.965,0.783], g[1.164]
>197, c[0.143,100], d[0.778,0.930], g[1.224]
>198, c[0.122,100], d[0.791,0.879], g[1.352]
>199, c[0.176,96], d[0.908,0.784], g[1.318]
>200, c[0.114,100], d[0.913,0.952], g[1.277]
>201, c[0.127,98], d[0.959,0.951], g[1.310]
>202, c[0.145,98], d[0.860,1.136], g[1.235]
>203, c[0.152,98], d[0.896,0.891], g[1.458]
>204, c[0.128,98], d[1.199,0.742], g[1.203]
>205, c[0.082,100], d[0.763,1.085], g[1.294]
>206, c[0.111,100], d[0.829,0.746], g[1.427]
>207, c[0.113,98], d[1.003,0.867], g[1.379]
>208, c[0.233,96], d[1.029,0.657], g[1.109]
>209, c[0.148,100], d[0.668,1.143], g[1.132]
>210, c[0.090,100], d[0.818,0.769], g[1.392]
>211, c[0.087,100], d[0.948,0.658], g[1.318]
>212, c[0.170,94], d[0.869,0.692], g[1.207]
>213, c[0.149,96], d[0.968,1.038], g[1.200]
>214, c[0.116,98], d[1.

>379, c[0.040,100], d[0.638,0.660], g[1.344]
>380, c[0.090,100], d[0.821,0.831], g[1.299]
>381, c[0.038,100], d[0.787,0.975], g[1.135]
>382, c[0.047,100], d[0.827,1.054], g[1.253]
>383, c[0.043,100], d[0.752,0.667], g[1.301]
>384, c[0.091,100], d[0.739,0.966], g[1.480]
>385, c[0.072,100], d[1.015,0.750], g[1.332]
>386, c[0.035,100], d[1.013,1.060], g[1.189]
>387, c[0.046,100], d[0.756,0.752], g[1.435]
>388, c[0.062,100], d[0.930,0.628], g[1.309]
>389, c[0.035,100], d[0.841,0.692], g[1.185]
>390, c[0.033,100], d[0.756,0.834], g[1.196]
>391, c[0.043,100], d[0.689,0.791], g[1.334]
>392, c[0.071,100], d[0.679,1.010], g[1.062]
>393, c[0.057,100], d[0.621,0.620], g[1.420]
>394, c[0.048,100], d[0.985,0.667], g[1.318]
>395, c[0.052,100], d[0.883,0.937], g[0.993]
>396, c[0.060,100], d[0.749,0.821], g[1.035]
>397, c[0.053,100], d[0.693,0.831], g[1.537]
>398, c[0.052,100], d[0.998,0.733], g[1.350]
>399, c[0.040,100], d[0.996,0.887], g[1.158]
>400, c[0.050,100], d[0.666,0.895], g[1.233]
>401, c[0.

...

>11128, c[0.000,100], d[0.867,0.743], g[1.299]
>11129, c[0.001,100], d[0.637,0.849], g[1.120]
>11130, c[0.001,100], d[0.750,0.851], g[1.333]
>11131, c[0.001,100], d[0.872,0.750], g[1.261]
>11132, c[0.002,100], d[0.946,0.811], g[1.148]
>11133, c[0.000,100], d[0.830,0.689], g[1.168]
>11134, c[0.001,100], d[0.578,0.622], g[1.179]
>11135, c[0.001,100], d[0.687,0.937], g[1.255]
>11136, c[0.000,100], d[0.823,0.800], g[1.162]
>11137, c[0.001,100], d[0.763,0.898], g[1.111]
>11138, c[0.001,100], d[0.795,1.048], g[1.106]
>11139, c[0.001,100], d[0.943,1.081], g[1.089]
>11140, c[0.000,100], d[0.865,0.840], g[1.205]
>11141, c[0.001,100], d[1.048,1.009], g[1.303]
>11142, c[0.001,100], d[0.767,0.879], g[1.106]
>11143, c[0.001,100], d[0.726,0.818], g[1.146]
>11144, c[0.001,100], d[0.804,0.947], g[1.302]
>11145, c[0.001,100], d[1.088,0.833], g[1.198]
>11146, c[0.001,100], d[0.753,0.917], g[1.131]
>11147, c[0.000,100], d[0.577,0.795], g[1.082]
>11148, c[0.001,100], d[0.813,0.887], g[1.213]
>11149, c[0.0

>11304, c[0.001,100], d[0.855,0.623], g[1.102]
>11305, c[0.002,100], d[0.712,0.826], g[1.179]
>11306, c[0.001,100], d[0.832,1.197], g[1.178]
>11307, c[0.001,100], d[0.743,0.649], g[1.087]
>11308, c[0.000,100], d[0.979,1.075], g[1.200]
>11309, c[0.001,100], d[0.815,0.971], g[1.018]
>11310, c[0.000,100], d[0.810,0.791], g[1.329]
>11311, c[0.002,100], d[0.940,1.028], g[1.165]
>11312, c[0.001,100], d[0.962,0.793], g[1.260]
>11313, c[0.001,100], d[0.937,0.802], g[1.062]
>11314, c[0.000,100], d[0.676,0.825], g[1.003]
>11315, c[0.001,100], d[0.842,1.010], g[0.951]
>11316, c[0.001,100], d[0.639,0.776], g[1.152]
>11317, c[0.001,100], d[0.847,0.705], g[1.199]
>11318, c[0.001,100], d[0.980,0.959], g[1.005]
>11319, c[0.002,100], d[0.878,0.984], g[0.984]
>11320, c[0.000,100], d[0.738,0.919], g[1.167]
>11321, c[0.000,100], d[0.606,0.784], g[1.067]
>11322, c[0.001,100], d[0.752,0.876], g[1.121]
>11323, c[0.001,100], d[0.698,0.926], g[1.235]
>11324, c[0.001,100], d[1.007,0.749], g[0.980]
>11325, c[0.0

>11480, c[0.003,100], d[0.554,0.920], g[1.139]
>11481, c[0.003,100], d[0.826,0.644], g[1.077]
>11482, c[0.001,100], d[0.612,0.853], g[1.178]
>11483, c[0.001,100], d[0.690,0.951], g[1.159]
>11484, c[0.001,100], d[0.727,1.305], g[1.126]
>11485, c[0.001,100], d[0.782,1.104], g[1.374]
>11486, c[0.000,100], d[0.757,0.827], g[1.263]
>11487, c[0.001,100], d[1.166,0.894], g[1.307]
>11488, c[0.000,100], d[1.029,0.869], g[1.114]
>11489, c[0.000,100], d[0.942,0.747], g[0.984]
>11490, c[0.000,100], d[0.839,0.882], g[1.009]
>11491, c[0.001,100], d[0.702,1.152], g[1.103]
>11492, c[0.001,100], d[0.686,0.859], g[1.235]
>11493, c[0.001,100], d[0.766,0.890], g[1.147]
>11494, c[0.001,100], d[0.608,1.096], g[1.289]
>11495, c[0.000,100], d[0.950,1.162], g[1.137]
>11496, c[0.002,100], d[0.806,1.105], g[1.069]
>11497, c[0.002,100], d[0.792,0.809], g[1.196]
>11498, c[0.000,100], d[0.833,1.078], g[1.205]
>11499, c[0.002,100], d[1.003,1.033], g[1.199]
>11500, c[0.004,100], d[0.973,0.973], g[1.161]
>11501, c[0.0

>11657, c[0.000,100], d[0.851,0.853], g[1.221]
>11658, c[0.001,100], d[0.766,0.880], g[1.142]
>11659, c[0.000,100], d[0.834,0.819], g[1.139]
>11660, c[0.002,100], d[0.538,0.959], g[1.243]
>11661, c[0.001,100], d[0.854,0.798], g[1.123]
>11662, c[0.000,100], d[0.621,0.770], g[1.223]
>11663, c[0.001,100], d[0.978,0.608], g[1.116]
>11664, c[0.000,100], d[0.721,0.718], g[1.109]
>11665, c[0.001,100], d[0.865,1.054], g[1.122]
>11666, c[0.000,100], d[0.981,0.856], g[1.264]
>11667, c[0.001,100], d[1.050,0.969], g[0.940]
>11668, c[0.000,100], d[0.750,1.193], g[1.038]
>11669, c[0.001,100], d[0.931,1.085], g[1.184]
>11670, c[0.001,100], d[0.782,0.921], g[1.215]
>11671, c[0.003,100], d[0.847,0.925], g[1.184]
>11672, c[0.001,100], d[0.669,0.839], g[1.108]
>11673, c[0.000,100], d[0.830,0.990], g[1.168]
>11674, c[0.001,100], d[0.597,1.057], g[1.204]
>11675, c[0.000,100], d[0.877,0.972], g[1.189]
>11676, c[0.001,100], d[1.093,0.857], g[1.244]
>11677, c[0.001,100], d[0.883,1.019], g[1.148]
>11678, c[0.0

>11832, c[0.001,100], d[0.742,0.787], g[1.202]
>11833, c[0.000,100], d[0.885,0.978], g[1.329]
>11834, c[0.001,100], d[0.839,0.933], g[1.350]
>11835, c[0.001,100], d[0.983,1.034], g[1.099]
>11836, c[0.000,100], d[0.730,1.065], g[1.276]
>11837, c[0.001,100], d[0.884,0.789], g[1.204]
>11838, c[0.001,100], d[0.905,1.113], g[1.202]
>11839, c[0.001,100], d[0.817,0.868], g[1.238]
>11840, c[0.000,100], d[0.702,0.860], g[1.371]
>11841, c[0.001,100], d[0.862,0.759], g[1.197]
>11842, c[0.001,100], d[0.815,0.824], g[1.191]
>11843, c[0.001,100], d[0.956,0.909], g[1.085]
>11844, c[0.001,100], d[0.869,0.782], g[1.010]
>11845, c[0.001,100], d[0.922,1.125], g[1.087]
>11846, c[0.000,100], d[0.715,0.931], g[1.141]
>11847, c[0.001,100], d[0.690,0.899], g[1.175]
>11848, c[0.002,100], d[0.960,0.861], g[1.132]
>11849, c[0.000,100], d[0.780,0.594], g[1.132]
>11850, c[0.000,100], d[0.854,1.036], g[1.113]
>11851, c[0.001,100], d[0.827,0.744], g[1.186]
>11852, c[0.000,100], d[0.768,0.819], g[0.952]
>11853, c[0.0